In [1]:
import re
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf


from keras.datasets import imdb

from keras.preprocessing import sequence

max_features = 5000
maxlen = 400
batch_size = 50
embedding_dims = 50
n_filters = 250
kernel_size = 3
hidden_dims = 250
learning_rate = 0.003
training_steps = 2
width = 3

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [3]:
a = sequence.pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]])
print(a)
print(y_train[:5])

[[0 1 2 3]
 [3 4 5 6]
 [0 0 7 8]]
[1 0 0 1 0]


In [4]:
X = tf.placeholder("int32", [batch_size, maxlen])
Y = tf.placeholder("float32", [batch_size,]) 

In [5]:
xavier_init = tf.contrib.layers.xavier_initializer()
word_embs  = tf.Variable(xavier_init([max_features, embedding_dims]))
xavier_init = tf.contrib.layers.xavier_initializer()
filters = tf.Variable(xavier_init([width, embedding_dims, n_filters]))

In [6]:
def neural_net(x):
    #print(tf.shape(x))
    
    x = tf.nn.embedding_lookup(word_embs, x) #embedding - returns batch x len x emb ????
    #print(tf.shape(x))
    x = tf.layers.dropout(inputs = x, rate = 0.2)
    #print(x.dtype)
    x = tf.nn.conv1d(value=x, filters = filters, stride = 1,padding = 'VALID')
    x = tf.nn.relu(features = x)
    #print(tf.shape(x))
    x = tf.layers.max_pooling1d(inputs = x, pool_size = maxlen - 2, strides = 1, padding = 'VALID') #pool_size = tf.shape(x)[1]
    #print(tf.shape(x))
    x = tf.squeeze(x, [1]) #because of one fictional dim from global max pooling
    x = tf.layers.dense(inputs = x, units = 250, activation = 'relu') #has his variables inside?
    x = tf.layers.dropout(inputs = x, rate = 0.2)
    #print(tf.shape(x))
    x = tf.layers.dense(inputs = x, units = 1)
    #x = tf.nn.sigmoid(x)
    #print(tf.shape(x))
    return x

In [7]:
logits = tf.squeeze(neural_net(X), [1])

#this WAS in LSTM. needed? 
prediction_sm = tf.nn.sigmoid(logits) # THIS IS THE ANSWER ALREADY


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits, labels=Y)) #there is softmax applied to the last output
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)



# Evaluate model (with test logits, for dropout to be disabled)
#print('\n',tf.shape(prediction), tf.shape(tf.argmax(prediction, 1)), tf.shape(tf.argmax(Y, 1)))
correct_pred = tf.equal(prediction_sm, Y)#tf.argmax(logits, 1),




accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [8]:
sess = tf.InteractiveSession()
sess.run(init)

#print(len(train_Y))
for step in range(1, training_steps+1):
    i = 0
    while i*batch_size < len(x_train):
        
        x_batch, y_batch = x_train[i*batch_size:(i+1)*batch_size], y_train[i*batch_size:(i+1)*batch_size]
        i += 1
        #print(i, x_batch.shape, y_batch.shape)
        
        sess.run(train_op, feed_dict={X: x_batch, Y: y_batch}) 
        
        #oss, acc = sess.run([loss_op, accuracy], feed_dict={X: x_batch, Y: y_batch}) #x_lookup
        #    
        #    print("Step " + str(step) + ", Minibatch Loss= " + \
        #          "{:.4f}".format(loss) + ", Training Accuracy= " + \
        #          "{:.3f}".format(acc))

print("Optimization Finished!")


Optimization Finished!


In [9]:
print(i)

500


In [17]:
prediction = np.array([])

i = 0
while i*batch_size < len(x_test):
    x_batch, y_batch = x_test[i*batch_size:(i+1)*batch_size], y_test[i*batch_size:(i+1)*batch_size]
    i += 1

    
    a = sess.run(prediction_sm, feed_dict={X: x_batch, Y: y_batch})
    #print(a,t)
    prediction = np.append(prediction, np.asarray(a)) #no need to flatten
    
prediction = [int(round(t)) for t in prediction]

TP, FP, FN = {0:0, 1:0}, {0:0, 1:0}, {0:0, 1:0}
for l in range(2):
    for counter in range(len(prediction)):
        if prediction[counter] == y_test[counter] and prediction[counter] == l: #answer matches label
            TP[l] += 1
        if prediction[counter] != y_test[counter] and prediction[counter] == l:
            FP[l] += 1
        if prediction[counter] != y_test[counter] and prediction[counter] != l:
            FN[l] += 1
            
Prec, Rec = {}, {}
for l in range(2):
    try:
        Prec[l] = TP[l]/float((TP[l]+FP[l]))
    except Exception:
        Prec[l] = 0
    try:
        Rec[l] = TP[l]/float((TP[l]+FN[l]))
    except Exception:
        Rec[l] = 0
F1 = {}
for l in range(2):
    try:
        F1[l] = 2*Prec[l]*Rec[l]/(Prec[l]+Rec[l])
    except Exception:
        F1[l] = 0
    
    
MF1 = sum(_ for _ in F1.values())/2
print(F1, MF1)

{0: 0.8801612107254482, 1: 0.8865441520012459} 0.883352681363347


In [11]:
print(prediction)

[0.01761236 0.986368   0.97832263 ... 0.01595182 0.12709025 0.33885211]


In [12]:
print(y_test)

[0 1 1 ... 0 0 0]


In [16]:
h = 0.9999

print(int(round(h)))

1
